In [1]:
import sys, os, inspect
sys.path.append("/home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL")

In [2]:
from tv.util.qubole_hive_client import QuboleHiveClient
from tv.util.s3_utils import S3
from tv import path_resolver
from tv.ds_metadata import S3Location

In [3]:
p = path_resolver.resolve('')
p

'/home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/'

In [4]:
!ls /home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/state/original/state.json

/home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/state/original/state.json


In [5]:
!ls /home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/state/working/state.json

/home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/state/working/state.json


In [20]:
path = os.path.join("state", 'working','state.json')
resolved_path = path_resolver.resolve(path)
s3_loc = 's3://tv-mta/tv-results-dev/state/'   #S3Location.tv_etl_states_loc
res = S3.s3_ftp(s3_loc, resolved_path, 'UPLOAD', debug=True, print_stderr=True)
print(resolved_path)
print(s3_loc)
assert res is not None and res[0] != '' , 'Saving working state file failed!'
res

stdout:
Completed 295 Bytes/295 Bytes (2.6 KiB/s) with 1 file(s) remaining
upload: ../state/working/state.json to s3://tv-mta/tv-results-dev/state/state.json

stderr:

/home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/state/working/state.json
s3://tv-mta/tv-results-dev/state/


['Completed 295 Bytes/295 Bytes (2.6 KiB/s) with 1 file(s) remaining',
 'upload: ../state/working/state.json to s3://tv-mta/tv-results-dev/state/state.json',
 '']

In [13]:
import re
assert re.match('^.*state/$',s3_loc) is not None, 'State location should have leaf dir state/'
a = re.match('^.*state/$',s3_loc)
a

<re.Match object; span=(0, 33), match='s3://tv-mta/tv-results-dev/state/'>

In [8]:
S3.list_s3_files('s3://tv-mta/tv-results-dev/')
#S3.list_s3_files_kantar('s3://tv-mta/kantar/tvweekly/')

stdout:
_tmp.mta-tv-impressions-2/
_tmp.mta-tv-impressions/
mta-tv-impressions-2/
mta-tv-impressions/
state/

stderr:



['_tmp.mta-tv-impressions-2/',
 '_tmp.mta-tv-impressions/',
 'mta-tv-impressions-2/',
 'mta-tv-impressions/',
 'state/',
 '']

In [59]:
!aws s3 rm s3://tv-mta/tv-results-dev/state --recursive

delete: s3://tv-mta/tv-results-dev/state/state.json


In [16]:
len(res)

1

In [61]:
path = os.path.join("state", 'original','state.json')
resolved_path = path_resolver.resolve(path)
s3_loc = os.path.join('s3://tv-mta/tv-results-dev/state/','state.json')  #S3Location.tv_etl_states_loc
res = S3.s3_ftp(s3_loc, resolved_path, 'UPLOAD', debug=True, print_stderr=True)
print(resolved_path)
print(s3_loc)
res

stdout:
Completed 225 Bytes/225 Bytes (2.0 KiB/s) with 1 file(s) remaining
upload: ../state/original/state.json to s3://tv-mta/tv-results-dev/state/state.json

stderr:

/home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/state/original/state.json
s3://tv-mta/tv-results-dev/state/state.json


['Completed 225 Bytes/225 Bytes (2.0 KiB/s) with 1 file(s) remaining',
 'upload: ../state/original/state.json to s3://tv-mta/tv-results-dev/state/state.json',
 '']

In [25]:
assert re.match('^.*state/state.json$',s3_loc) is not None, 'State location should have leaf dir state/state.json'

In [30]:
path = os.path.join("state", 'template','state.json')
templete_file_path = path_resolver.resolve(path)
path = os.path.join("state", 'original','state.json')
original_file_path = path_resolver.resolve(path)
path = os.path.join("state", 'working','state.json')
working_file_path = path_resolver.resolve(path)

from shutil import copyfile

copyfile(templete_file_path, working_file_path)

'/home/vbhargava/fb_v9.0_merged/msaction_backend/common/BU3.0_core/util/Py_utils/ETL/state/working/state.json'

In [33]:
import json
def read(path = None):
    if path is None:
        path = os.path.join("state", 'working','state.json') 
    resolved_path = path_resolver.resolve(path)
    with open(resolved_path, 'r') as qubole_config_file:       
        state_dict  = json.load(qubole_config_file)
    return state_dict

In [34]:
data_json = read()

In [42]:
data_json['tv_etl_history'][0]['started_on']='asc'


In [43]:
data_json['tv_etl_history']

[{'status': 'start',
  'started_on': 'asc',
  'finished_on': '',
  'tv_start_date': '',
  'tv-end-date': '',
  'for_noOfDays': -1}]

In [46]:
def write(data_dict: dict, path = None):
    if path is None:
        path = os.path.join("state", 'working','state.json') 
    resolved_path = path_resolver.resolve(path)
    with open(resolved_path, 'w') as outfile:       
        json.dump(data_dict, outfile, indent=4)


In [57]:
write(data_json)

In [53]:
model : dict = {}
model['status'] = "InProgress"
model['started_on'] = "started_on"
model['finished_on'] = "finished_on"
model['tv-start-date'] = "cls.status"
model['tv-end-date'] = "cls.status"
model['trigger_no_of_days'] = "cls.status"
l = data_json['tv_etl_history']
l

[{'status': 'start',
  'started_on': 'asc',
  'finished_on': '',
  'tv_start_date': '',
  'tv-end-date': '',
  'for_noOfDays': -1}]

In [55]:
l.insert(0, model)

In [56]:
data_json

{'optimistic_lock': '',
 'tv_etl_history': [{'status': 'InProgress',
   'started_on': 'started_on',
   'finished_on': 'finished_on',
   'tv-start-date': 'cls.status',
   'tv-end-date': 'cls.status',
   'trigger_no_of_days': 'cls.status'},
  {'status': 'start',
   'started_on': 'asc',
   'finished_on': '',
   'tv_start_date': '',
   'tv-end-date': '',
   'for_noOfDays': -1}]}

In [13]:
df = QuboleHiveClient.hive_query(query="SELECT min(t.file_name) as end_date from tv_mta_staging_dev2.ext_raw_tivo_log_2 t where t.file_name >= '20190107';")

In [23]:
str(df['end_date'][0]).isnumeric()

True

In [12]:
df1 = QuboleHiveClient.hive_query(query="SELECT max(t.file_name) as end_date from tv_mta_staging_dev2.ext_raw_tivo_log_2 t where t.file_name <= '20190107' and  t.file_name >='20190101';")

In [20]:
df1['end_date'][0].isnumeric()

False

In [24]:
df2 = QuboleHiveClient.hive_query(query="select distinct week from tv_mta_staging_dev2.raw_kantar_log_2 where year = 2019 and week = 36;")
df2

,week
0,36


In [25]:
df2['week'][0]

36